In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
#%matplotlib osx    
import orbital
import math

In [2]:
def Newtons_Equation(t, X):
    r = X[0:3]
    v = X[3:6]

    
    F = -(G/au*year) * m_p9 * m_sun /np.linalg.norm(r)**3 * r
    a    = F / m_p9
    
    return np.concatenate((v, a))

In [3]:
# // Constants //
year  = 3.154e+7 #s
au = 1.496e+8 #km
m_earth = 5.972e24 #kg
m_p9 = 10 * m_earth #fiducial 10 M⊕ Planet Nine candidate proposed by Batygin & Brown (2016)
m_sun = 333000 * m_earth
M = m_sun + m_p9
G  = 6.674e-20  #km^3/kg s^2
mu = G /au*year * M


# //// Fienga et al 2016 ///
i_p9 = 30 #degrees
Omega_p9 = 113 #degrees
argp_p9 = 150 #degrees
nu_p9 = 118 #degrees

a_p9 = 700 * au #km
e_p9 = 0.6

In [4]:
# // Calculate r = [U, V, W] using orbital angles
#https://pythonhosted.org/OrbitalPy/modules/utilities/?highlight=orbital.utilities.uvw_from_elements#orbital.utilities.uvw_from_elements

U, V, W = orbital.utilities.uvw_from_elements(math.radians(i_p9), raan=math.radians(Omega_p9), arg_pe=math.radians(argp_p9), f=nu_p9)
x = np.linalg.norm(U)
y = np.linalg.norm(V)
z = np.linalg.norm(W)
r = np.array([x,y,z])
print(r)

[1. 1. 1.]


In [5]:
# calculate heliocentric distance using semi major axis and eccentricity
a = a_p9
e = e_p9
nu = nu_p9
r_p9 = (a * (1 - e**2)) / (1 + e*np.cos(nu))
print('r_p9 = ', r_p9, 'km', 'or ', r_p9/au, 'au')

p_p9 = np.sqrt((a/au)**3)
print('p_p9 = ', p_p9, 'Years')

v_p9 = np.sqrt(G*M*((2 / r_p9) - (1 / a)))
print('v_p9 = ', v_p9, 'km/s', 'or ', v_p9/au, 'au/s', 'or ', v_p9/au*year, 'au/year')

r_p9 =  60190520986.67269 km or  402.3430547237479 au
p_p9 =  18520.259177452135 Years
v_p9 =  1.7727977105891672 km/s or  1.1850252076130796e-08 au/s or  0.3737569504811653 au/year


In [6]:
#perihelia roughly between 150 and 350 au, semimajor axis between 380 and 980 au
#Brown, Batygin "Observational constraints on the orbit and location of Planet Nine in the outer solar system."
#a = 700

apocenter_radius = 550 #au
pericenter_radius = 150 #au
orbital.utilities.elements_for_apsides(apocenter_radius, pericenter_radius)

(350.0, 0.5714285714285714)

In [7]:
#// User Inputs // 

x = a_p9/au #au 
y = 0
z = 0
vx = 0
vy = v_p9/au*year # au/year
vz = 0

r = np.array([x,y,z])
v = np.array([vx,vy,vz])

print('r = ', r)
print('v = ', v)

r =  [700.   0.   0.]
v =  [0.         0.37375695 0.        ]


In [8]:
#/// Calculate orbital elements /// 
#follows from the method laid out in Fundamentals of Astrodynamics and Applications, by Vallado, 2007
# https://space.stackexchange.com/questions/1904/how-to-programmatically-calculate-orbital-elements-using-position-velocity-vecto

h = np.cross(r,v)         # angular momentum
K = np.array([0, 0, 1])
nhat=np.cross(K,h)       #node vector

#eccentricity vector
evec = ((np.linalg.norm(v)**2 - mu/np.linalg.norm(r))*r - np.dot(r,v)*v)/ mu  
# evec = (np.linalg.norm(abs(v))**2/mu - 1/np.linalg.norm(abs(r)))*r - (np.dot(r,v)*v / mu) 

#evec = np.array([e_p9, 0, 0])
e = np.linalg.norm(evec)

if e == 0:
    orbit = 'circular orbit'
if e > 0 and e < 1:
    orbit = 'elliptical orbit'
if e == 1:
    orbit ='parabolic orbit'
if e > 1:
    orbit = 'hyperbolic orbit'
print('e = ', e, orbit)

energy = np.linalg.norm(v)**2/2 - mu/np.linalg.norm(r) #Specific mechanical energy
if energy < 0: #absolute value of potential energy is larger than kinetic
    bound = 'bound'
if energy > 0: #kinetic energy is larger than the absolute value of the potential energy
    bound = 'unbound'
print('E = ', energy, bound)

if abs(e) != 1: 
    a = -mu/(2*energy)
    #p = a*(1-e**2)
    p = np.sqrt(abs(a)**3)
else:
    #p = np.linalg.norm(h)**2/mu
    a = 'inf'
    p = 'inf'
    
print('a = ', a)         #semi major axis


print('p = ', p)        #period

i = np.arccos(h[2]/np.linalg.norm(h))  #inclination
print('i = ', i)        

Omega = np.arccos(nhat[0]/np.linalg.norm(nhat))  #swivel: the angle from the principal direction to the ascending node, 0° ≤ Ω < 360°
if nhat[1] < 0:
   Omega = 360-Omega
print('Omega = ', Omega)

argp = np.arccos(np.dot(nhat,evec)/(np.linalg.norm(nhat)*e)) # argument of perigee, ω: 0° ≤ ω < 360
if evec[2]<0:
   argp = 360-argp
print('argp = ', argp)

#nu_0 = nu_p9
nu_0 = np.arccos(np.dot(evec,r)/(e*np.linalg.norm(r)))  # True anomaly, ν, is the angle along the orbital path from perigee to the spacecraft’s position vector r.  0 <= v < 360°
if np.dot(r,v)<0:
   nu = 360 - nu_0
print('initial nu = ', nu_0, 'degrees')  # changes with time, location of the spacecraft in its orbit

e =  0.9999999965055145 elliptical orbit
E =  -39975631.33190336 bound
a =  350.000000611535
p =  6547.900444015555
i =  0.0
Omega =  nan
argp =  nan
initial nu =  3.141592653589793 degrees


/Users/CEE/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
/Users/CEE/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in double_scalars


In [9]:
dt = .001
time = np.arange(0,10.001, dt)
tmax = 10
tt = []
t = 0

In [10]:
xt = []
yt = []
zt = []
vx = []
vy = []
vz = []

X = np.concatenate((r,v))

%matplotlib osx  

while True:

    r = X[0:3]
    v = X[3:6]
    
    
    xt.append(X[0])
    yt.append(X[1])
    zt.append(X[2])
    
    
    #X = X + Newtons_Equation(X) * h    #EULERS METHOD
    
    f1 = Newtons_Equation(t       ,X          )    #RUNGE KUTTA METHOD
    f2 = Newtons_Equation(t+dt/2.0,X+f1*dt/2.0)
    f3 = Newtons_Equation(t+dt/2.0,X+f2*dt/2.0)
    f4 = Newtons_Equation(t+dt    ,X+f3*dt    )

    X = X + (f1 + 2.0*f2 + 2.0*f3 + f4) / 6.0 * dt
    
    nu = np.arccos(np.dot(evec,r)/(e*np.linalg.norm(r)))  # True anomaly, ν, is the angle along the orbital path from perigee to the spacecraft’s position vector r.  0 <= v < 360°
    if np.dot(r,v)<0:
        nu = 360 - nu
    #print('nu = ', nu)
    if nu *10000 //1 / 10000 == (nu_0-.001)*10000 //1 / 10000:
        break
    
    tt.append(t)

    if t >= tmax:
        break
    t += dt

In [11]:
print(t)
print(p_p9)
print(len(yt))

10.000999999999896
18520.259177452135
10002


In [12]:
# // 2D orbit plot // 
plt.plot(xt, yt, 'r-')
plt.xlabel('X axis')
plt.ylabel('Y axis')
plt.plot(0,0,'*',mfc='w',ms=10)
#plt.gca().set_aspect('equal', adjustable='box')  
plt.show()

In [14]:
#// 3D orbit plot //
XT = np.array(xt)
YT = np.array(yt)
ZT = np.array(zt)


plt.rcParams['legend.fontsize'] = 10

fig = plt.figure(figsize=(5,5))
ax = fig.gca(projection='3d')
ax.set_aspect("equal")


ax.plot(XT, YT, ZT, label='Orbit', antialiased=False)     #orbit
#ax.text(0, 0, 0, '*', zdir=None)


u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:100j]   #sphere
x = 0.1 * np.cos(u)*np.sin(v)
y = 0.1 * np.sin(u)*np.sin(v)
z = 0.1 * np.cos(v)
ax.plot_surface(x, y, z, color="b")


ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_zlim(-1.5, 1.5)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

plt.draw()
plt.pause(0.005)
plt.show()
